In [7]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import random

### Make a list of the urls of the categories in the forum

In [5]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:4]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news']

### Get the discussion topic urls for the first 10 pages of the category

In [10]:
link_listdiscussion = []
for i in range(1, 11):
    url = f'{categories[3]}/p{i}'
    #print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    time.sleep(random.randint(10,120))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

# Loop through all the topics found for each catagory        
list_for_mongo = []
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
    time.sleep(random.randint(10,120))
    

list_for_mongo

[{'signature': 'Rosy :compress:\n\n3A/High Porosity/Medium Density\n\n'},
 {'signature': 'Fran\nNaturallyCurly Graphic Designer'},
 {'signature': '3a/3b high porosity, mainly coarse, medium density\nLive in Canada- hot summers, freezing winters- dry dry dry\nAs I am cleansing pudding, soultanicals deep condish, curlsmith cowash, lhdc condish\xa0\nKCCC, curlsmith soufflé, ufdcm, mga sculpting gel\xa0\nScrunch with hair repear towel, then diffuse or air\nProtein- Cnpf'},
 {'signature': 'Gretchen\nNaturallyCurly.com co-founder\n3A\n\nYou are beautiful!'},
 {'signature': '$5 off first order at iherb use code KEY066'},
 {'signature': 'Happy, Nappy, and Kinky 24/7!!\n4/b very coily'},
 {'signature': '\nMod CG-Naturally Highly Porous and Dry\n\xa0*This list is not exhaustive...\nCoWash: various\nDT: various\nLeave-in: 100% Mafura butter, Shima Hair Leave-in\xa0\nStylers: MoroccanOil Curl Defining Cream, variousSpritz/Sealant:not now\n*experimenting and loving it!'},
 {'signature': 'Wurly, fin

### Create a mongoDB and have data stored from scrape 

In [12]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [13]:
entry= curly_collection.insert_many(list_for_mongo)

BulkWriteError: batch op errors occurred

In [14]:
query_1 = curly_collection.find({})
for x in query_1:
    print(x)

{'_id': ObjectId('5d4761a20bd621c34eabc2b7'), 'signature': "Animal obsessed wavy with a love for writing\xa02B/2C Indian wavy, low-medium porosity, medium-high density, coarse hairShampoo: WOW Miracle 10-in-1 ShampooConditioner: WOW Argan Oil ConditionerGel: Arata Hair GelDC/ Treatment: Shea Moisture Manuka Honey and Mafura Oil Intensive Hydration MasqueClarifying Shampoo: Pantene Lively CleanCG since May '18After nearly two decades of neglect, CG finally helped me embrace my hair as a part and parcel of my identity"}
{'_id': ObjectId('5d4761a20bd621c34eabc2b8'), 'signature': '2c/3a | low porosity | fine/medium | low/medium density| protein,coconut, and green tea sensitive | Modified CGMLow-Poo:\xa0 Jessicurl Hair Cleansing CreamSulfate Wash: Aveda Rosemary Mint Purifying ShampooRinse Out Conditioner: DevalCurl One ConditionLeave-In Conditioner:\xa0Curl Junkie Smoothing LotionDeep Conditioner:\xa0DevaCurl Heaven In HairCurl Enhancer: Curl Junkie Spiral LotionGel: Jessicurl\xa0Spiralici